In [2]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Activation, UpSampling2D, Conv2D
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
from PIL import Image
import matplotlib.pyplot as plt
import sys
import os
import numpy as np
import cv2
import glob

In [3]:
def build_generator():
    noise_shape = (100,)
    noise = Input(shape=noise_shape)
    
    model = Sequential()
    model.add(Dense(64 * 4 * 4,activation="relu", input_shape=noise_shape))
    model.add(Reshape((4,4,64)))
    model.add(BatchNormalization(momentum=0.8))
    
    for layer in [128,256,128,64]:
        model.add(UpSampling2D())
        model.add(Conv2D(layer, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2D(16, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Conv2D(channels, kernel_size=3, padding="same"))
    model.add(Activation("tanh"))

    model.summary()

    output_img = model(noise)

    return Model(noise, output_img)

In [4]:
def build_discriminator():
    img = Input(shape=img_size)
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_size, padding="same")) #32,32
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    
    for layer in [64,128,128,128]:
        model.add(Conv2D(layer, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    model.summary()
    validity = model(img)
    return Model(img, validity)

In [5]:
def build_model():
    # Build and compile the generator
    generator = build_generator()
    generator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002,0.5))

    # The generator takes noise as input and generated imgs
    noise = Input(shape=(100,))
    generated_image = generator(noise)

    # Build and compile the discriminator
    discriminator = build_discriminator()
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002,0.5), metrics=['accuracy'])
    discriminator.trainable = False

    # The valid takes generated images as input and determines validity
    validity = discriminator(generated_image)

    gan = Model(noise, validity)
    gan.compile(loss='binary_crossentropy', optimizer=Adam(0.0002,0.5))
    
    return discriminator,generator,gan

In [6]:
def load_dataset(batch_size, image_shape, data_dir=None):
    sample_dim = (batch_size,) + image_shape
    sample = np.empty(sample_dim, dtype=np.float32)
    all_data_dirlist = list(glob.glob(data_dir))
    sample_imgs_paths = np.random.choice(all_data_dirlist,batch_size)
    for index,img_filename in enumerate(sample_imgs_paths):
        image = Image.open(img_filename)
        image = image.resize(image_shape[:-1])
        image = np.asarray(image)
        image = (image/127.5) -1 # normalizing data
        sample[index,...] = image
    return sample

def read_data():
    image_shape=(64,64,3)
    X_train = load_dataset(30000, (64,64,3), "data/*.png")
    print('data loaded')
    return X_train

In [7]:
def train(epochs, batch_size=128, save_interval=50):
    global discriminator, generator, gan
    half_batch = int(batch_size / 2)
    for epoch in range(epochs):

        #  Train Discriminator
        real_ids = np.random.randint(0, X_train.shape[0], half_batch)
        real_imgs = X_train[real_ids]

        noise = np.random.normal(0, 1, (half_batch, 100))
        # Generate a half batch of new images
        gen_imgs = generator.predict(noise)

        # Train the discriminator
        real_loss = discriminator.train_on_batch(real_imgs, np.ones((half_batch, 1)))
        fake_loss = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(real_loss, fake_loss)

        #  Train Generator
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % save_interval == 0:
            save_imgs(epoch)

def save_imgs(epoch):
    global discriminator, generator, gan
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[i*c+j])
            axs[i,j].axis('off')
            
    fig.savefig("results/animes_%d.png" % epoch, dpi=600, bbox_inches='tight')
    plt.close()    
    generator.save("models/generator_%d.h5" % epoch)
    discriminator.save("models/discriminator_%d.h5" % epoch)

In [8]:
img_rows = 64
img_cols = 64
channels = 3

img_size = (img_rows, img_cols, channels)

In [ ]:
X_train = read_data()

In [ ]:
print X_train.shape

In [9]:
discriminator, generator, gan = build_model()

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1024)              103424    
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 64)          0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 4, 4, 64)          256       
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 128)         0         
________________________________________________

In [ ]:
train(epochs=100000, batch_size=256, save_interval=200)

In [ ]:
noise = np.random.normal(0, 1, (1,100))
gan_img = generator.predict(noise)
print(discriminator.predict(gan_img))
img = 0.5*gan_img[0] + 0.5
plt.imshow(img)
plt.show()

In [ ]:
plt.imshow(img)
plt.axis('off')
plt.savefig('generated_anime.png', dpi=300, bbox_inches='tight')